# ZS Data Science Challenge

In [121]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Import and analyse the data

In [122]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [123]:
train.head()

,S_No,Year,Month,Week,Merchant_ID,Product_ID,Country,Sales
0,1,2013,1,1,ar00001,1,Argentina,157500.0
1,2,2013,1,1,ar00003,1,Argentina,39375.0
2,3,2013,1,1,ar00004,1,Argentina,15750.0
3,4,2013,1,1,ar00007,1,Argentina,47250.0
4,5,2013,1,1,ar00008,1,Argentina,283500.0


In [124]:
test.head()

,S_No,Year,Month,Product_ID,Country,Sales
0,79073,2016,4,1,Argentina,NaN
1,79074,2016,5,1,Argentina,NaN
2,79075,2016,6,1,Argentina,NaN
3,79076,2016,7,1,Argentina,NaN
4,79077,2016,8,1,Argentina,NaN


In [125]:
train = pd.DataFrame(train)
test = pd.DataFrame(test)

In [126]:
train.shape,test.shape

((79072, 8), (105, 6))

In [127]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79072 entries, 0 to 79071
Data columns (total 8 columns):
S_No           79072 non-null int64
Year           79072 non-null int64
Month          79072 non-null int64
Week           79072 non-null int64
Merchant_ID    79072 non-null object
Product_ID     79072 non-null int64
Country        79072 non-null object
Sales          79072 non-null float64
dtypes: float64(1), int64(5), object(2)
memory usage: 4.8+ MB


In [128]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 6 columns):
S_No          105 non-null int64
Year          105 non-null int64
Month         105 non-null int64
Product_ID    105 non-null int64
Country       105 non-null object
Sales         0 non-null float64
dtypes: float64(1), int64(4), object(1)
memory usage: 5.0+ KB


In [129]:
# we should drop the features week and merchant id from the training data set,
#because it is missing in test data  set
train = train.drop(['S_No','Week','Merchant_ID'],axis = 1)

In [130]:
train.head()

,Year,Month,Product_ID,Country,Sales
0,2013,1,1,Argentina,157500.0
1,2013,1,1,Argentina,39375.0
2,2013,1,1,Argentina,15750.0
3,2013,1,1,Argentina,47250.0
4,2013,1,1,Argentina,283500.0


## Encoding for the feature 'Country'

In [131]:
#So all features are non-null in train.csv
#we need to find promising features for our model
from sklearn.preprocessing import LabelEncoder
var_mod = ['Country']
le = LabelEncoder()
for i in var_mod:
    train[i] = le.fit_transform(train[i])
train.dtypes



Year            int64
Month           int64
Product_ID      int64
Country         int64
Sales         float64
dtype: object

In [132]:
# create a copy of test data set so that the test dataframe doesnt get disturbed much and then we can 
# directly assign the predictions to the test data

In [133]:
a = test.copy(deep = True)

In [134]:
#a = a.drop(['Sales','S_No'],axis = 1).copy(deep = True)


In [135]:
from sklearn.preprocessing import LabelEncoder
var_mod = ['Country']
le = LabelEncoder()
for i in var_mod:
    a[i] = le.fit_transform(a[i])
train.dtypes

Year            int64
Month           int64
Product_ID      int64
Country         int64
Sales         float64
dtype: object

In [136]:
test.head()
# just to cross validate that whether test data is same as before or not

,S_No,Year,Month,Product_ID,Country,Sales
0,79073,2016,4,1,Argentina,NaN
1,79074,2016,5,1,Argentina,NaN
2,79075,2016,6,1,Argentina,NaN
3,79076,2016,7,1,Argentina,NaN
4,79077,2016,8,1,Argentina,NaN


In [137]:
test['Country'].unique()

array(['Argentina', 'Belgium', 'Columbia', 'Denmark', 'England',
       'Finland'], dtype=object)

Change the train data frame such that it gives information about sales for each month for a particular country
in a specific year

In [138]:
train= train.groupby(['Country','Year','Month','Product_ID']).agg({'Sales': np.sum})


In [139]:
train = train.reset_index()
train.head()

,Country,Year,Month,Product_ID,Sales
0,0,2013,1,1,34346025.00
1,0,2013,1,2,2751851.48
2,0,2013,2,1,32005575.00
3,0,2013,2,2,2804313.12
4,0,2013,3,1,32530050.00


We are also going to consider the feature expense_per_promotion for each product  

In [140]:
expense = pd.read_csv('promotional_expense.csv')
expense.head(5)

,Year,Month,Country,Product_Type,Expense_Price
0,2013,1,Argentina,1,14749.307
1,2013,1,Argentina,2,1329.374
2,2013,1,Belgium,2,249.590
3,2013,1,Columbia,1,1893.122
4,2013,1,Columbia,2,1436.726


In [141]:
#same encoding of country feature for expense data is performed

In [142]:
from sklearn.preprocessing import LabelEncoder
var_mod = ['Country']
le = LabelEncoder()
for i in var_mod:
    expense[i] = le.fit_transform(expense[i])
train.dtypes

Country         int64
Year            int64
Month           int64
Product_ID      int64
Sales         float64
dtype: object

In [143]:
expense.columns = ['Year','Month','Country','Product_ID','Expense']

Now we will merge expense data set and our current dataset, but for test data there is no feature for expense 
so we will add the feature in its copied dataframe, then use it for prediction and then assign those inclusive sales to test dataset

In [144]:
a = pd.merge(a, expense, on=['Country','Year','Month','Product_ID'] ,how='outer')
a

,S_No,Year,Month,Product_ID,Country,Sales,Expense
0,79073.0,2016,4,1,0,NaN,8214.875
1,79074.0,2016,5,1,0,NaN,10777.878
2,79075.0,2016,6,1,0,NaN,10320.673
3,79076.0,2016,7,1,0,NaN,7377.587
4,79077.0,2016,8,1,0,NaN,9805.705
5,79078.0,2016,9,1,0,NaN,9375.481
6,79079.0,2016,10,1,0,NaN,7284.900
7,79080.0,2016,11,1,0,NaN,10381.438
8,79081.0,2016,12,1,0,NaN,6865.382
9,79082.0,2017,1,1,0,NaN,10704.394


In [145]:
a['Expense'].fillna(np.mean(a['Expense']),inplace = True)

In [146]:
train = pd.merge(train, expense, on=['Country','Year','Month','Product_ID'] ,how='inner')


In [147]:
train

,Country,Year,Month,Product_ID,Sales,Expense
0,0,2013,1,1,34346025.00,14749.307
1,0,2013,1,2,2751851.48,1329.374
2,0,2013,2,1,32005575.00,12187.566
3,0,2013,2,2,2804313.12,1315.006
4,0,2013,3,1,32530050.00,13076.579
5,0,2013,3,2,2573004.98,1192.655
6,0,2013,4,1,35588700.00,14377.199
7,0,2013,4,2,3436237.42,1658.633
8,0,2013,5,1,38789100.00,15652.861
9,0,2013,5,2,3772468.84,1838.573


In [148]:
expense.head(5)
#once check whether it is properly merged or not

,Year,Month,Country,Product_ID,Expense
0,2013,1,0,1,14749.307
1,2013,1,0,2,1329.374
2,2013,1,1,2,249.590
3,2013,1,2,1,1893.122
4,2013,1,2,2,1436.726


In [149]:
# Extract features and labels
labels = train['Sales']
features = train.drop('Sales', axis = 1)

# List of features for later use
feature_list = list(features.columns)

# Convert to numpy arrays
import numpy as np

features = np.array(features)
labels = np.array(labels)

In [150]:
#split train and cross vaildation(test) data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features,labels,test_size=0.33,
                                                    random_state=123)

In [154]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import roc_auc_score

In [155]:
#using random forests
rf= RandomForestRegressor(n_estimators = 1000,random_state = 42)
rf.fit(X_train, y_train);
prediction = rf.predict(X_test)
rf.score(X_train,y_train)

0.998760577881276

In [157]:
from sklearn import metrics
from sklearn.metrics import explained_variance_score

score = explained_variance_score(y_test,prediction)
score

0.9826892997497877

In [158]:
from sklearn.metrics import r2_score
r2_score(y_test,prediction)

0.9826126217199911

In [159]:
errors = abs(prediction - y_test)
errors
mape = np.mean(100 * (errors / y_test))
mape
accuracy = 100 - mape
print('Accuracy:', round(accuracy, 2), '%.')
#print('Average absolute error:', round(np.mean(errors), 2), 'degrees.')


Accuracy: 90.07 %.


In [163]:
a['Sales'] = rf.predict(a.drop(['Sales','S_No'],axis = 1))

In [164]:
b = test.copy(deep=True)
# craete another copy of test data to merge a with the new copy and then without 
# affecting features of test data we will put the values for sales

In [165]:
from sklearn.preprocessing import LabelEncoder
var_mod = ['Country']
le = LabelEncoder()
for i in var_mod:
    b[i] = le.fit_transform(b[i])
train.dtypes

Country         int64
Year            int64
Month           int64
Product_ID      int64
Sales         float64
Expense       float64
dtype: object

In [166]:
b.drop(['Sales'],axis = 1,inplace = True)


In [167]:
b = pd.merge(b, a, on=['S_No','Year','Month','Product_ID','Country'] ,how='inner')


In [168]:
b

,S_No,Year,Month,Product_ID,Country,Sales,Expense
0,79073,2016,4,1,0,3.528882e+07,8214.8750
1,79074,2016,5,1,0,3.478572e+07,10777.8780
2,79075,2016,6,1,0,3.528514e+07,10320.6730
3,79076,2016,7,1,0,3.552944e+07,7377.5870
4,79077,2016,8,1,0,3.528514e+07,9805.7050
5,79078,2016,9,1,0,3.528514e+07,9375.4810
6,79079,2016,10,1,0,3.552944e+07,7284.9000
7,79080,2016,11,1,0,3.528514e+07,10381.4380
8,79081,2016,12,1,0,3.573222e+07,6865.3820
9,79082,2017,1,1,0,3.528514e+07,10704.3940


In [169]:
test['Sales'] = b['Sales']

In [170]:
test

,S_No,Year,Month,Product_ID,Country,Sales
0,79073,2016,4,1,Argentina,3.528882e+07
1,79074,2016,5,1,Argentina,3.478572e+07
2,79075,2016,6,1,Argentina,3.528514e+07
3,79076,2016,7,1,Argentina,3.552944e+07
4,79077,2016,8,1,Argentina,3.528514e+07
5,79078,2016,9,1,Argentina,3.528514e+07
6,79079,2016,10,1,Argentina,3.552944e+07
7,79080,2016,11,1,Argentina,3.528514e+07
8,79081,2016,12,1,Argentina,3.573222e+07
9,79082,2017,1,1,Argentina,3.528514e+07


In [171]:
#Convert test dataframe to csv file such that we can use it afterwards
test.to_csv('submission2018_update1.csv',index = False)